In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn.model_selection import KFold, cross_val_score

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("Irisnew.csv")
train.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,-1
1,2,4.9,3.0,1.4,0.2,-1
2,3,4.7,3.2,1.3,0.2,-1
3,4,4.6,3.1,1.5,0.2,-1
4,5,5.0,3.6,1.4,0.2,-1


In [3]:
train=np.asarray(train)
X=np.zeros((150,5))
y=np.zeros((150,1))
X=np.asarray(X)
y=np.asarray(y)
train.shape

(300, 6)

In [4]:
k=1
sig=0.5
r=0
m=np.zeros((150,5))
m=np.asarray(m)

In [5]:
print(np.shape(m))

(150, 5)


In [6]:
 count=0

In [7]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    if(y_test[0]==1):
         test_min=test_min+1
    else:
        test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    if(y_predict[0]==1 and y_predict[0]==y_test[0]):
         pred_min=pred_min+1
    elif(y_predict[0]==-1 and y_predict[0]==y_test[0]):
         pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)

In [8]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y,m):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None
        m=np.asarray(m)
        self.K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K1[i][j] = gaussian_kernel(m[i], X[j])
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [9]:

if __name__ == "__main__":
    import pylab as pl  
   
    def test_soft():
        count=0
        for v in range (0,150):
            for i in range (0,150):
                for j in range(5):
                    X[i][j]=train[i+v][j]
                    y[i]=train[i+v][5]
                for j in range(5):
                    m[i][j]=((X[i][j]-(k*r*sig))+(X[i][j]+(k*(2-r)*sig)))/2
                    #rint(m[i][j])
            y_train=np.zeros((149))
            y_test=np.zeros((1))
            for i in range(0,149):
                y_train[i]=y[i]
            y_test[0]=y[149]
            X_train=np.zeros((149,5))
            X_test=np.zeros((1,5))
            for i in range(0,149):
                for j in range(0,5):
                    X_train[i][j]=X[i][j]
            for j in range (0,5):
                X_test[0][j]=X[149][j]

            clf = SVM(C=100.0)
            clf.fit(X_train, y_train,m)

            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            if(correct==1):
                count=count+1
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print(correct/len(y_predict))
            print(count)
        print("%d no of misclassification"%(150-count))
        print("Probability of misclassification",(150-count)/150)

    test_soft()
    #test_non_linear()
    

(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2611e+04 -1.3552e+05  2e+05  2e-13  3e-14
 1:  8.1739e+03 -1.4334e+04  2e+04  1e-13  3e-14
 2:  1.1544e+03 -1.5692e+03  3e+03  9e-14  2e-14
 3:  1.4002e+02 -2.4242e+02  4e+02  3e-14  7e-15
 4:  7.2767e+00 -4.5856e+01  5e+01  4e-15  2e-15
 5: -6.9816e+00 -1.4921e+01  8e+00  5e-15  8e-16
 6: -8.4103e+00 -9.8688e+00  1e+00  1e-15  3e-16
 7: -8.6153e+00 -8.9855e+00  4e-01  4e-15  2e-16
 8: -8.6959e+00 -8.8271e+00  1e-01  5e-15  2e-16
 9: -8.7324e+00 -8.7661e+00  3e-02  3e-15  3e-16
10: -8.7429e+00 -8.7486e+00  6e-03  6e-15  2e-16
11: -8.7445e+00 -8.7452e+00  6e-04  4e-15  2e-16
12: -8.7447e+00 -8.7448e+00  3e-05  2e-15  3e-16
13: -8.7448e+00 -8.7448e+00  6e-07  5e-15  3e-16
Optimal solution found.
optimal
(149,)
53 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
1
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2219e+04 -1.3553e+05  2e

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
10
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2604e+04 -1.3623e+05  2e+05  1e-13  3e-14
 1:  8.1830e+03 -1.4386e+04  2e+04  6e-14  3e-14
 2:  1.1559e+03 -1.5716e+03  3e+03  4e-14  1e-14
 3:  1.4019e+02 -2.4281e+02  4e+02  1e-15  7e-15
 4:  7.2745e+00 -4.5934e+01  5e+01  8e-16  2e-15
 5: -6.9984e+00 -1.4949e+01  8e+00  3e-15  7e-16
 6: -8.4272e+00 -9.8897e+00  1e+00  5e-15  3e-16
 7: -8.6332e+00 -9.0053e+00  4e-01  2e-15  2e-16
 8: -8.7147e+00 -8.8511e+00  1e-01  7e-16  2e-16
 9: -8.7506e+00 -8.7849e+00  3e-02  1e-15  2e-16
10: -8.7608e+00 -8.7680e+00  7e-03  2e-15  2e-16
11: -8.7630e+00 -8.7634e+00  3e-04  4e-16  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  4e-16  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  4e-15  3e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
11

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
20
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2585e+04 -1.3621e+05  2e+05  2e-14  3e-14
 1:  8.1806e+03 -1.4381e+04  2e+04  5e-14  3e-14
 2:  1.1558e+03 -1.5716e+03  3e+03  5e-14  2e-14
 3:  1.4017e+02 -2.4278e+02  4e+02  5e-14  7e-15
 4:  7.2698e+00 -4.5932e+01  5e+01  9e-16  2e-15
 5: -7.0010e+00 -1.4950e+01  8e+00  7e-15  9e-16
 6: -8.4301e+00 -9.8878e+00  1e+00  2e-15  3e-16
 7: -8.6345e+00 -9.0045e+00  4e-01  6e-16  2e-16
 8: -8.7161e+00 -8.8472e+00  1e-01  1e-15  2e-16
 9: -8.7511e+00 -8.7845e+00  3e-02  2e-15  2e-16
10: -8.7608e+00 -8.7678e+00  7e-03  6e-16  2e-16
11: -8.7630e+00 -8.7634e+00  4e-04  5e-16  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  2e-16  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  2e-15  2e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
21

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
29
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2586e+04 -1.3622e+05  2e+05  1e-13  3e-14
 1:  8.1806e+03 -1.4381e+04  2e+04  2e-15  4e-14
 2:  1.1558e+03 -1.5716e+03  3e+03  5e-14  2e-14
 3:  1.4017e+02 -2.4278e+02  4e+02  3e-15  7e-15
 4:  7.2702e+00 -4.5932e+01  5e+01  2e-15  2e-15
 5: -7.0005e+00 -1.4949e+01  8e+00  2e-15  8e-16
 6: -8.4292e+00 -9.8889e+00  1e+00  1e-16  3e-16
 7: -8.6339e+00 -9.0048e+00  4e-01  2e-15  2e-16
 8: -8.7150e+00 -8.8450e+00  1e-01  2e-15  2e-16
 9: -8.7509e+00 -8.7844e+00  3e-02  5e-16  2e-16
10: -8.7608e+00 -8.7678e+00  7e-03  4e-16  2e-16
11: -8.7630e+00 -8.7633e+00  3e-04  2e-15  3e-16
12: -8.7632e+00 -8.7632e+00  1e-05  3e-15  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  2e-15  2e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
30

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
38
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2600e+04 -1.3622e+05  2e+05  1e-13  3e-14
 1:  8.1787e+03 -1.4368e+04  2e+04  5e-14  4e-14
 2:  1.1553e+03 -1.5705e+03  3e+03  4e-14  2e-14
 3:  1.4011e+02 -2.4267e+02  4e+02  7e-15  8e-15
 4:  7.2689e+00 -4.5910e+01  5e+01  2e-15  2e-15
 5: -6.9966e+00 -1.4942e+01  8e+00  2e-15  8e-16
 6: -8.4269e+00 -9.8856e+00  1e+00  1e-15  3e-16
 7: -8.6337e+00 -9.0063e+00  4e-01  2e-15  2e-16
 8: -8.7143e+00 -8.8460e+00  1e-01  3e-16  3e-16
 9: -8.7507e+00 -8.7843e+00  3e-02  2e-16  2e-16
10: -8.7608e+00 -8.7679e+00  7e-03  2e-16  3e-16
11: -8.7630e+00 -8.7634e+00  3e-04  1e-15  3e-16
12: -8.7632e+00 -8.7632e+00  1e-05  2e-15  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  1e-15  3e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
39

...

y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
103
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2700e+04 -1.3624e+05  2e+05  2e-13  3e-14
 1:  8.1954e+03 -1.4399e+04  2e+04  1e-13  3e-14
 2:  1.1576e+03 -1.5737e+03  3e+03  2e-14  2e-14
 3:  1.4041e+02 -2.4310e+02  4e+02  3e-14  7e-15
 4:  7.3008e+00 -4.5981e+01  5e+01  3e-15  2e-15
 5: -6.9970e+00 -1.4958e+01  8e+00  1e-15  8e-16
 6: -8.4287e+00 -9.8918e+00  1e+00  4e-16  3e-16
 7: -8.6340e+00 -9.0058e+00  4e-01  3e-16  2e-16
 8: -8.7149e+00 -8.8466e+00  1e-01  3e-16  2e-16
 9: -8.7507e+00 -8.7845e+00  3e-02  8e-16  2e-16
10: -8.7608e+00 -8.7679e+00  7e-03  7e-16  2e-16
11: -8.7630e+00 -8.7634e+00  3e-04  1e-15  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  5e-16  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  1e-15  2e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
1

y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
112
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2706e+04 -1.3628e+05  2e+05  5e-14  3e-14
 1:  8.1967e+03 -1.4403e+04  2e+04  1e-13  3e-14
 2:  1.1578e+03 -1.5740e+03  3e+03  6e-14  2e-14
 3:  1.4044e+02 -2.4314e+02  4e+02  3e-15  7e-15
 4:  7.3029e+00 -4.5988e+01  5e+01  1e-15  2e-15
 5: -6.9974e+00 -1.4960e+01  8e+00  5e-15  7e-16
 6: -8.4293e+00 -9.8925e+00  1e+00  1e-15  3e-16
 7: -8.6342e+00 -9.0061e+00  4e-01  3e-16  2e-16
 8: -8.7145e+00 -8.8462e+00  1e-01  3e-16  2e-16
 9: -8.7503e+00 -8.7838e+00  3e-02  2e-15  2e-16
10: -8.7607e+00 -8.7679e+00  7e-03  7e-16  3e-16
11: -8.7630e+00 -8.7634e+00  3e-04  3e-15  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  9e-16  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  4e-16  2e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
1

(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2706e+04 -1.3629e+05  2e+05  3e-13  3e-14
 1:  8.1968e+03 -1.4403e+04  2e+04  2e-13  3e-14
 2:  1.1578e+03 -1.5740e+03  3e+03  3e-14  2e-14
 3:  1.4044e+02 -2.4315e+02  4e+02  7e-15  7e-15
 4:  7.3025e+00 -4.5990e+01  5e+01  3e-15  2e-15
 5: -6.9980e+00 -1.4960e+01  8e+00  2e-15  8e-16
 6: -8.4299e+00 -9.8932e+00  1e+00  2e-15  3e-16
 7: -8.6347e+00 -9.0069e+00  4e-01  6e-16  2e-16
 8: -8.7147e+00 -8.8464e+00  1e-01  2e-15  1e-16
 9: -8.7508e+00 -8.7846e+00  3e-02  2e-15  2e-16
10: -8.7608e+00 -8.7679e+00  7e-03  1e-15  2e-16
11: -8.7630e+00 -8.7634e+00  3e-04  8e-16  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  2e-15  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  2e-15  3e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
122
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2700e+04 -1.3623e+05  

y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
131
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2703e+04 -1.3626e+05  2e+05  8e-15  3e-14
 1:  8.1961e+03 -1.4401e+04  2e+04  1e-13  3e-14
 2:  1.1577e+03 -1.5738e+03  3e+03  7e-14  1e-14
 3:  1.4043e+02 -2.4312e+02  4e+02  3e-14  7e-15
 4:  7.3018e+00 -4.5985e+01  5e+01  7e-15  2e-15
 5: -6.9973e+00 -1.4959e+01  8e+00  3e-15  7e-16
 6: -8.4291e+00 -9.8923e+00  1e+00  4e-16  3e-16
 7: -8.6342e+00 -9.0060e+00  4e-01  7e-16  2e-16
 8: -8.7146e+00 -8.8463e+00  1e-01  2e-15  2e-16
 9: -8.7507e+00 -8.7844e+00  3e-02  2e-15  2e-16
10: -8.7608e+00 -8.7678e+00  7e-03  8e-16  2e-16
11: -8.7630e+00 -8.7634e+00  3e-04  6e-16  2e-16
12: -8.7632e+00 -8.7632e+00  1e-05  2e-15  2e-16
13: -8.7632e+00 -8.7632e+00  4e-07  1e-15  3e-16
Optimal solution found.
optimal
(149,)
51 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
1

y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
141
(149, 149)
     pcost       dcost       gap    pres   dres
 0:  4.2710e+04 -1.3629e+05  2e+05  3e-13  3e-14
 1:  8.1972e+03 -1.4404e+04  2e+04  3e-13  3e-14
 2:  1.1578e+03 -1.5740e+03  3e+03  4e-15  1e-14
 3:  1.4045e+02 -2.4315e+02  4e+02  5e-15  7e-15
 4:  7.3044e+00 -4.5990e+01  5e+01  6e-15  2e-15
 5: -6.9968e+00 -1.4960e+01  8e+00  2e-15  7e-16
 6: -8.4286e+00 -9.8917e+00  1e+00  1e-15  3e-16
 7: -8.6335e+00 -9.0054e+00  4e-01  1e-15  2e-16
 8: -8.7142e+00 -8.8460e+00  1e-01  3e-15  2e-16
 9: -8.7506e+00 -8.7843e+00  3e-02  5e-15  2e-16
10: -8.7609e+00 -8.7677e+00  7e-03  5e-15  2e-16
11: -8.7630e+00 -8.7633e+00  3e-04  3e-15  2e-16
12: -8.7631e+00 -8.7632e+00  1e-05  2e-15  2e-16
13: -8.7631e+00 -8.7632e+00  4e-07  4e-15  2e-16
Optimal solution found.
optimal
(149,)
50 support vectors out of 149 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
1